# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [1]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots
using LaTeXStrings

In [2]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [3]:
# load data
# ni = niread("data/nifti_dt.nii.gz") * 1e9;
ni = niread("../../../data/nifti_dt_nonlinear.nii.gz") * 1e9;
size(ni)

(112, 112, 50, 1, 6)

In [4]:
 d1, d2, d3, _, d = size(ni)

# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [ # data ordered as [xx, yx, yy, zx, zy, zz]
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];
    
# pick a 1D slice 
data = vec(predata[34:81,32:79,15]); # TODO just get all the data (that is non-zero) and put it in a 1D array!!!!
print(size(data))
# construct data manifold
n = size(data)[1] 
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M);

(2304,)

In [5]:
# Export slice image
num_export = 20
asymptote_export_SPD("results/Camino1D_orig.asy", data=data[1:min(num_export,n)], scale_axes=(2,2,2));

### Construct low rank approximation ###

In [6]:
q = mean(M, data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [11]:
curvature_corrected_low_rank_approximation(M, q, data, 1; stepsize=1/500, max_iter=20, change_tol=1e-4)

Initial  F(x): 4.22480413859 | 
# 1     change: 6.217888522 |  F(x): 1.91786435802 | 
# 2     change: 3.492781408 |  F(x): 1.14104844733 | 
# 3     change: 2.213168748 |  F(x): 0.82422564498 | 
# 4     change: 1.458206643 |  F(x): 0.68455166339 | 
# 5     change: 0.998577254 |  F(x): 0.61804268548 | 
# 6     change: 0.709030714 |  F(x): 0.58406626587 | 
# 7     change: 0.518616694 |  F(x): 0.56570495184 | 
# 8     change: 0.387680803 |  F(x): 0.55537309279 | 
# 9     change: 0.294079954 |  F(x): 0.54940111850 | 
# 10    change: 0.225170845 |  F(x): 0.54589012204 | 
# 11    change: 0.173404291 |  F(x): 0.54380434096 | 
# 12    change: 0.134005914 |  F(x): 0.54255740028 | 
# 13    change: 0.103777288 |  F(x): 0.54180910397 | 
# 14    change: 0.080469806 |  F(x): 0.54135901367 | 
# 15    change: 0.062445304 |  F(x): 0.54108791053 | 
# 16    change: 0.048481238 |  F(x): 0.54092447538 | 
# 17    change: 0.037651052 |  F(x): 0.54082589458 | 
# 18    change: 0.029245762 |  F(x): 0.54076641201

([[24.42970402293359 -0.33964141672890874 -0.50245473645762; -0.33964141672890874 23.001784263029407 -0.27683708016370806; -0.50245473645762 -0.27683708016370806 22.358081386982935]], [0.0011084770115784015; -0.00181408779184214; … ; 0.00036078830219510305; -0.0032671954126559525;;])

In [20]:
# exact_low_rank_approximation(M, q, data, 1; stepsize=1/1000000000, max_iter=2); 

In [10]:
max_iter = 5 # maybe just do 50 here to show that initially the exact solver does a bit better, but for low rank matrices, we can use our method

nR_q = []
nU = []
ccR_q = []
ccU = []
eR_q = []
eU = []
for i in 1:d # this takes at least 2500 minutes (= 4h 10min)...
    println("#$(i) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_rank_approximation(M, q, data, i)
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_rank_approximation(M, q, data, i; stepsize=1/1000, max_iter=max_iter, change_tol=1e-4); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q, eUr = exact_low_rank_approximation(M, q, data, i; stepsize=1/10000000000, max_iter=max_iter, change_tol=1e-4); 
    push!(eR_q, eRr_q)
    push!(eU, eUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
Initial  F(x): 4.22480413859 | 
# 1     change: 3.108944261 |  F(x): 2.89145801401 | 
# 2     change: 2.390343755 |  F(x): 2.09255521438 | 
# 3     change: 1.889382028 |  F(x): 1.58915356240 | 
# 4     change: 1.519486073 |  F(x): 1.26174255757 | 
# 5     change: 1.236136249 |  F(x): 1.04416885967 | 
The algorithm reached its maximal number of iterations (5).
#1 | computing exact low-rank approximation
Initial  F(x): 0.53834899660 | 
# 1     change: 0.002272143 |  F(x): 0.53834927893 | 
# 2     change: 0.002271281 |  F(x): 0.53834955454 | 
# 3     change: 0.002270415 |  F(x): 0.53834982342 | 
# 4     change: 0.002269545 |  F(x): 0.53835008553 | 
# 5     change: 0.002268672 |  F(x): 0.53835034087 | 
The algorithm reached its maximal number of iterations (5).
#2 | computing naive low-rank approximation
#2 | computing curvature corrected low-rank approximation
Initial  F(x): 10.8674347116

In [26]:
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(d)
predicted_naive_distances_r= zeros(d)
true_naive_distances_r= zeros(d)

curvature_corrected_tangent_distances_r = zeros(d)
predicted_curvature_corrected_distances_r = zeros(d)
true_curvature_corrected_distances_r = zeros(d)

exact_tangent_distances_r = zeros(d)
exact_distances_r= zeros(d)

for rank in 1:d
    naive_log_q_data_r = Symmetric.([sum([nR_q[rank][i] * nU[rank][k,i] for i in 1:rank]) for k in 1:n])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[rank][i] * ccU[rank][k,i] for i in 1:rank]) for k in 1:n])
    exact_log_q_data_r = Symmetric.([sum([eR_q[rank][i] * eU[rank][k,i] for i in 1:rank]) for k in 1:n])
    
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)
    exact_data_r = exp.(Ref(M), Ref(q), exact_log_q_data_r)

    # compute relative tangent space error
    naive_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance
    exact_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - exact_log_q_data_r).^2) / ref_distance

    # compute relative manifold error
    predicted_naive_distances_r[rank] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[rank] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[rank] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[rank] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
    exact_distances_r[rank] = exact_loss(M, q, data, exact_log_q_data_r)
end

In [27]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
plot(1:d, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r, exact_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/Camino1D_errors_by_rank.png")
plot(1:d, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16, exact_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino1D_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/Camino1D_logerrors_by_rank.png"

In [28]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(1:d, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/Camino1D_discrepancy_by_rank.png")
plot(1:d, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino1D_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/Camino1D_logdiscrepancy_by_rank.png"